In [36]:
%run ItemModel.ipynb
%run APIService.ipynb

In [4]:
import inspect, re
import pandas as pd
import csv

# Class GetData

In [37]:
class GetData(apiServiceDelegate):
    newest = 0
    items_json = []
    all_items = [] # List SHOPPEItem
    itemModel = SHOPPEItem()
    fileName = 'ShoppeItem.tsv'
    APIService = APIService(newest, keyword = "thoi trang nam", categoryids = "2829")
    
    i = 1
    def __init__(self, newest):
        self.newest = newest
        self.APIService.delegate = self
        pass
    
    def setNewest(self,newest):
        self.newest = newest
        pass
    
    def getData(self):
        while True:
            try:
                self.APIService.setNewest(self.newest)
                check = self.APIService.get()
                if check == False: break
                
            except Exception as e:
                self.parseData()
                print(f'Error: {e}')
                break
        pass
    
    def parseData(self):
        self.all_items.clear()
        for item in self.items_json:
            itemModel = SHOPPEItem()
            itemModel.name = item['name']
            itemModel.shop_location = item['shop_location']
            itemModel.ctime = item['ctime'] #time since 1970
            itemModel.time_now = int(time.time())
            itemModel.sell_time = itemModel.time_now - itemModel.ctime
            itemModel.historical_sold = item['historical_sold']
            itemModel.sold = item['sold']
            itemModel.shopee_verified = item['shopee_verified']
            itemModel.discount = item['discount']
            itemModel.is_adult = item['is_adult']
            if len(item['tier_variations']) > 0:
                itemModel.options = 1
                for tier_variation in item['tier_variations']:
                    itemModel.options *= len(tier_variation['options'])
            else:
                itemModel.options = 0
            itemModel.show_official_shop_label_in_title = item['show_official_shop_label_in_title']
            itemModel.rating_star = item['item_rating']['rating_star']
            itemModel.rating_count = item['item_rating']['rating_count']
            itemModel.flash_sale = item['flash_sale']
            itemModel.upcoming_flash_sale = item['upcoming_flash_sale']
            itemModel.price_min = item['price_min']
            itemModel.price_min_before_discount = item['price_min_before_discount']
            
            itemModel.price_max = item['price_max']
            itemModel.price_max_before_discount = item['price_max_before_discount']
            itemModel.price = item['price']
            itemModel.price_before_discount = item['price_before_discount']
            itemModel.coin_earn_label = item['coin_earn_label']
            itemModel.liked_count = item['liked_count']
            itemModel.view_count = item['view_count']
            itemModel.cmt_count = item['cmt_count']
            itemModel.is_preferred_plus_seller = item['is_preferred_plus_seller']
            itemModel.show_free_shipping = item['show_free_shipping']
            itemModel.images = len(item['images'])
            itemModel.video_info_list = len(item['video_info_list'])
            self.all_items.append(itemModel)
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'name\tshop_location\titem_public_time\titem_be_got_time\tsell_time\tshopee_verified\tdiscount\tis_adult\t'
                + f'options\tshow_official_shop_label_in_title\trating_star\trating_count\t'
                + f'flash_sale\tupcoming_flash_sale\tprice_min\tprice_min_before_discoun\t'
                + f'price_max\tprice_max_before_discount\tprice\tprice_before_discount\t'
                + f'coin_earn_label\tliked_count\tview_count\tcmt_count\t'
                + f'is_preferred_plus_seller\tshow_free_shipping\timages\tvideo_info_list\t'
                + f'sold\thistorical_sold\n')

        for item in self.all_items:
                file.write(f'{item.name}\t{item.shop_location}\t{item.ctime}\t{item.time_now}\t{item.sell_time}\t{item.shopee_verified}\t{item.discount}\t{item.is_adult}\t'
                + f'{item.options}\t{item.show_official_shop_label_in_title}\t{item.rating_star}\t{item.rating_count}\t'
                + f'{item.flash_sale}\t{item.upcoming_flash_sale}\t{item.price_min}\t{item.price_min_before_discount}\t'
                + f'{item.price_max}\t{item.price_max_before_discount}\t{item.price}\t{item.price_before_discount}\t'
                + f'{item.coin_earn_label}\t{item.liked_count}\t{item.view_count}\t{item.cmt_count}\t'
                + f'{item.is_preferred_plus_seller}\t{item.show_free_shipping}\t{item.images}\t{item.video_info_list}\t'
                + f'{item.sold}\t{item.historical_sold}\n')
                
    def updateDataWhenSuccess(self, json_pydata):
        print(len(json_pydata))
        print(f'get! {self.i}')
        
        if json_pydata['items'] != None:
            self.items_json.extend(json_pydata['items'])
        else:
            self.parseData()
            print('get All Data!')
            return False

        self.newest += 50
        self.i += 1
        time.sleep(10)
        return True
        pass
    def getDataError(self, error):
        self.parseData()
        print(f'Error when request: {error}')
        return False
        pass
    
    pass

# Get Data

In [38]:
getData = GetData(0)

In [39]:
getData.getData()


Error when request: Expecting value: line 1 column 1 (char 0)


In [81]:
# getData.parseData()

In [10]:
# Check last request
getData.APIService.json_pydata

In [92]:
getData.APIService.headers


{'authority': 'shopee.vn',
 'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
 'x-shopee-language': 'vi',
 'x-requested-with': 'XMLHttpRequest',
 'if-none-match-': '55b03-3fd0f4bf7f1503951d3409cbf31998e0',
 'sec-ch-ua-mobile': '?0',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36 FS',
 'x-api-source': 'pc',
 'accept': '*/*',
 'sec-fetch-site': 'same-origin',
 'sec-fetch-mode': 'cors',
 'sec-fetch-dest': 'empty',
 'referer': 'https://shopee.vn/search?facet=1899&keyword=thoi%20trang%20nam&noCorrection=true&page=0',
 'accept-language': 'vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5',
 'cookie': 'SPC_IA=-1; SPC_F=xy8cDlkhDlqLeCmJKA43hs8UvmvwNAdV; REC_T_ID=527dc03e-e820-11ea-af33-3c15fb3aee63; _fbp=fb.1.1598503612222.1102617144; G_ENABLED_IDPS=google; SPC_CLIENTID=xy8cDlkhDlqLeCmJrfeazpkeljwtewtt; csrftoken=uBzcFbopA2X4oZMfFC2ZxJnd0pIMfdgr; welcomePkgShown=true; _hjid

In [93]:
# getData.APIService.url

In [11]:
# Test ParseData
print(getData.all_items[0].name)
print(getData.all_items[50].name)
print(getData.all_items[1].name)
print(getData.all_items[51].name)

Áo khoác nỉ cao cấp nam , áo khoác loại nhất thời trang nam mẫu MS1223
Áo Khoác Phao Cộc Tay Cổ Chữ V Thời Trang Nam
ÁO KHOÁC DÙ 2 LỚP HỌA TIẾT CÁ TÍNH, THỜI TRANG NAM ĐỘC ĐÁO CC
Áo Hoodies Có Nón Dài Tay Thời Trang Nam


In [30]:
getData.saveFile("ShoppeItem_thoiTrangNam_aoKhoacAoVest.tsv")

In [46]:
# Test saveFile
items_df = pd.read_csv('ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)

items_df



,Unnamed: 0,name,shop_location,item_public_time,item_be_got_time,sell_time,shopee_verified,discount,is_adult,options,...,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,sold,historical_sold
0,0,"[FREESHIP-HÀNG ĐẸP]Áo gió - Áo gió nam đẹp - Chất liệu poly chống nước, chống gió cực tốt, phù hợp đi đường dài",Hà Nội,1603690408,1609771212,6080804,False,34%,False,16,...,None,12,726,32,False,False,4,0,80,329
1,1,"Áo chống nắng nam 2020, Chất Vải kim cương sần Siêu Mát, logo thêu chữ",Hà Nội,1593582384,1609771212,16188828,True,44%,False,36,...,None,233,7343,719,False,True,9,1,311,2171
2,2,[MÃ GIẢM GIÁ 20K] Áo Khoác Nam Da Lộn Được Yêu Thích Nhất 2020,Hà Nội,1605514797,1609771212,4256415,False,39%,False,12,...,None,303,46025,144,False,False,5,1,229,369
3,3,[Video] Áo Nỉ Nam Thu Đông Freesize Cao Cấp Dưới 68kg Mặc Vừa,Hà Nội,1604312284,1609771212,5458928,True,3%,False,6,...,None,387,64191,139,False,True,9,1,1298,2958
4,4,"Áo gió lông cừu nam nhẹ chống nước, khoác dù ấm thời trang Hàn Quốc DRAWWIND V480",Hà Nội,1605256508,1609771212,4514704,True,49%,False,24,...,None,18,2006,61,False,False,7,0,76,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,945,"Áo Nỉ Hoodie Có Mũ Nhiều Màu, Áo Nỉ Thời Trang Nam.",Hà Nội,1543304845,1609771212,66466367,False,40%,False,3,...,None,4,6,3,False,True,1,1,0,9
946,946,Áo Khoác Khóa Kéo Thời Trang Nam,Nước ngoài,1602835676,1609771212,6935536,False,50%,False,16,...,None,0,0,0,False,False,7,0,0,1
947,947,"Áo khoác kaki cao cấp nam KIng168 , áo khoác 2 lớp thời trang nam MA01",TP. Hồ Chí Minh,1601353611,1609771212,8417601,False,32%,False,3,...,None,1,80,2,False,False,5,0,0,2
948,948,Áo khoác thời trang nam họa tiết rằn ri quân đội,Nước ngoài,1559101257,1609771212,50669955,True,20%,False,20,...,None,2,37,1,False,False,9,0,0,2


In [43]:
# Migrate Data
# Run this if you run the crawler before it've been updated to auto wirte the sell_time feature.
items_df.insert(4, 'sell_time', items_df['item_be_got_time'] - items_df['item_public_time'])
items_df.to_csv('ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep = '\t')

In [24]:
# Test time since 1970 for feature ctime, time_now
seconds = 0
seconds = time.time()
print("Seconds since epoch =", seconds)	

Seconds since epoch = 1609770842.0814402


In [ ]:
del items_df